In [25]:
import numpy as np
import pandas as pd
import math
from sklearn.model_selection import train_test_split
import sklearn.neighbors
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn import metrics
from sklearn.preprocessing import scale
from collections import Counter
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import mean_absolute_error 

In [2]:
headers = ["carat",	"cut","color","clarity","depth","table","price","x","y","z"]
data = pd.read_csv('diamonds.csv', na_values='?',    
         header=None,  names = headers) 
data = data.reset_index(drop=True)
data = data.iloc[1:]
data.describe()

,carat,cut,color,clarity,depth,table,price,x,y,z
count,53940,53940,53940,53940,53940,53940,53940,53940,53940,53940
unique,273,5,7,8,184,127,11602,554,552,375
top,0.3,Ideal,G,SI1,62,56,605,4.37,4.34,2.7
freq,2604,21551,11292,13065,2239,9881,132,448,437,767


In [3]:
#encode
map_cut = {"Fair":1, "Good":2, "Very Good" : 3, "Premium" : 4, "Ideal" : 5}
map_color = {"J": 1,"I":2, "H":3, "G":4, "F":5, "E":6, "D":7}
map_clarity = {"I1":1, "SI2":2, "SI1":3, "VS2":4, "VS1":5, "VVS2":6, "VVS1":7, "IF":8}

data["map_cut"] = data.cut.map(map_cut)
data["map_color"] = data.color.map(map_color)
data["map_clarity"] = data.clarity.map(map_clarity)

data.head()


,carat,cut,color,clarity,depth,table,price,x,y,z,map_cut,map_color,map_clarity
1,0.23,Ideal,E,SI2,61.5,55,326,3.95,3.98,2.43,5,6,2
2,0.21,Premium,E,SI1,59.8,61,326,3.89,3.84,2.31,4,6,3
3,0.23,Good,E,VS1,56.9,65,327,4.05,4.07,2.31,2,6,5
4,0.29,Premium,I,VS2,62.4,58,334,4.2,4.23,2.63,4,2,4
5,0.31,Good,J,SI2,63.3,58,335,4.34,4.35,2.75,2,1,2


In [4]:
data = data[:100]

In [5]:
data = data.drop(data.columns[[1,2,3]], axis=1) 

In [6]:
# data.isnull().sum()
# data = data.iloc[:10]
# # print("hah ", data.iloc[1])
# for row in data:
#     print (row)
#     for val in row:
#         val = int(val)
# data.dtypes
data = data.apply(pd.to_numeric) 
data.dtypes

carat          float64
depth          float64
table          float64
price            int64
x              float64
y              float64
z              float64
map_cut          int64
map_color        int64
map_clarity      int64
dtype: object

In [7]:
data["table"] = data['table'].astype(float) 
data.dtypes

carat          float64
depth          float64
table          float64
price            int64
x              float64
y              float64
z              float64
map_cut          int64
map_color        int64
map_clarity      int64
dtype: object

In [8]:
data.head(10)

,carat,depth,table,price,x,y,z,map_cut,map_color,map_clarity
1,0.23,61.5,55.0,326,3.95,3.98,2.43,5,6,2
2,0.21,59.8,61.0,326,3.89,3.84,2.31,4,6,3
3,0.23,56.9,65.0,327,4.05,4.07,2.31,2,6,5
4,0.29,62.4,58.0,334,4.20,4.23,2.63,4,2,4
5,0.31,63.3,58.0,335,4.34,4.35,2.75,2,1,2
6,0.24,62.8,57.0,336,3.94,3.96,2.48,3,1,6
7,0.24,62.3,57.0,336,3.95,3.98,2.47,3,2,7
8,0.26,61.9,55.0,337,4.07,4.11,2.53,3,3,3
9,0.22,65.1,61.0,337,3.87,3.78,2.49,1,6,4
10,0.23,59.4,61.0,338,4.00,4.05,2.39,3,3,5


In [9]:
#normalize
def normalize_data(col):
#     print ("col : ", col)
    max_val = col.max()
#     print ("mmax : ", max_val)
    min_val = col.min()
#     print("first _val : ", col[0])
    for i in range(len(col)):
#         print("val : ", col[i])
        col[i] = (col[i]-min_val)/(max_val-min_val)
#         print("afterval : ", col[i])
    return col
#     result = data.copy()
#     for feature_col in col:
#         max_value = data[feature_col].max()
#         min_value = data.columns[[feature_col]].min()
#         result.columns[[feature_col]] = (data.columns[[feature_col]] - min_value) / (max_value - min_value)
#     return result

In [10]:
col_to_normalize = ["carat","depth","table", "x","y", "z"]
for col in col_to_normalize:
    data[col] = normalize_data(data[col].values)
# result = normalize_data(col_to_normalize)

In [11]:
data.head(15)

,carat,depth,table,price,x,y,z,map_cut,map_color,map_clarity
1,0.039474,0.571429,0.066667,326,0.060150,0.089147,0.088889,5,6,2
2,0.013158,0.419643,0.466667,326,0.037594,0.034884,0.022222,4,6,3
3,0.039474,0.160714,0.733333,327,0.097744,0.124031,0.022222,2,6,5
4,0.118421,0.651786,0.266667,334,0.154135,0.186047,0.200000,4,2,4
5,0.144737,0.732143,0.266667,335,0.206767,0.232558,0.266667,2,1,2
6,0.052632,0.687500,0.200000,336,0.056391,0.081395,0.116667,3,1,6
7,0.052632,0.642857,0.200000,336,0.060150,0.089147,0.111111,3,2,7
8,0.078947,0.607143,0.066667,337,0.105263,0.139535,0.144444,3,3,3
9,0.026316,0.892857,0.466667,337,0.030075,0.011628,0.122222,1,6,4
10,0.039474,0.383929,0.466667,338,0.078947,0.116279,0.066667,3,3,5


In [12]:
def euclidean_distance(row1, row2):
    print ("row1 :", row1)
    print ("row2 :", row2)
    distance = 0.0
    for i in range(len(row1)-1):
        distance += (row1[i] - row2[i])**2
    return sqrt(distance)

In [13]:
# Separate X and y data
Y = data.price
X = data.drop('price', axis=1)
# Y

In [14]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25,random_state=1)
# print(X_train)
# print(Y_train)

In [15]:
# Changing the index of the records to sequential
X_train.index=range(len(X_train))
Y_train.index=range(len(X_train))
X_test.index=range(len(X_test))
Y_test.index=range(len(Y_test))

In [16]:
Y_train

0      402
1     2759
2      357
3      403
4      403
      ... 
70     554
71     338
72     554
73     342
74     402
Name: price, Length: 75, dtype: int64

In [17]:
# Function to return the list of distances of the test records from train records

#eucledian
def distNeighbours(X_train,Y_train,X_test,K):
    distance=[]
    for i in range(len(X_train)):
        eDistance=0
        for j in range(len(X_train.columns)):   
            eDistance+=round(np.sqrt(pow((X_train.iloc[i,j]-X_test[j]),2)),2)
        distance.append((eDistance,i,Y_train.iloc[i]))
        distance=sorted(distance, key=lambda x: x[0])[0:K]
    return distance


def distNeighbours(X_train,Y_train,X_test,K):
    distance=[]
    for i in range(len(X_train)):
        mDistance=0
        for j in range(len(X_train.columns)):   
            eDistance+=round(np.sqrt(pow((X_train.iloc[i,j]-X_test[j]),2)),2)
        distance.append((eDistance,i,Y_train.iloc[i]))
        distance=sorted(distance, key=lambda x: x[0])[0:K]
    return distance

# Manhattan 
def distNeighbours(X_train,Y_train,X_test,K):
    distance=[]
    for i in range(len(X_train)):
        mDistance=0
        for j in range(len(X_train.columns)):   
            mDistance+=round(abs(X_train.iloc[i,j]-X_test[j]),2)
        distance.append((mDistance,i,Y_train.iloc[i]))
        distance=sorted(distance, key=lambda x: x[0])[0:K]
    return distance

# Predict the output of the numeric variables based on K nearest neighbours
# Output is the mean of the K nearest neighbours
def predictOutputNumeric(X_train,Y_train,X_test,K):
    neighbours=[]
    responses=[]
    for i in range(len(X_test)):
        neighbours.append(distNeighbours(X_train,Y_train,X_test.iloc[i,:],K))
    for i in neighbours:
        mean=0
        for j in i:
            mean+=j[-1]
        mean=mean/K
        responses.append(mean)
    return responses

In [18]:
# Predict GPA
output=predictOutputNumeric(X_train,Y_train,X_test,5)

In [19]:
print ("output  : ", output)

output  :  [553.8, 523.6, 423.8, 523.4, 1806.4, 369.2, 389.6, 524.0, 553.8, 513.8, 1876.2, 410.8, 401.4, 422.6, 391.4, 423.8, 413.8, 422.6, 523.4, 369.2, 423.6, 510.4, 2758.8, 918.0, 374.6]


In [21]:
print ("output  : ", output)

output  :  [2819.75, 2790.75, 2831.25, 2827.5, 2835.5, 2810.0, 2838.25, 2824.75, 2808.5, 478.5, 2851.25, 2860.25, 2827.5, 2810.5, 2804.0, 2844.0, 2829.5, 500.25, 2839.75, 2829.5, 2804.25, 2841.75, 2799.75, 2804.25, 2831.25, 2810.0, 2810.0, 2848.75, 2795.25, 2811.25, 2831.25, 2822.5, 402.25, 2826.0, 479.0, 2818.25, 1110.75, 555.25, 2828.75, 2870.0, 2841.0, 2806.25, 2827.75, 2847.75, 2856.0, 2257.0, 2843.0, 1052.75, 2836.25, 2813.0, 2854.0, 391.0, 2829.5, 390.75, 2826.5, 2827.0, 2864.25, 2825.0, 2826.75, 2817.75, 2812.25, 2843.0, 478.75, 556.0, 2852.0, 2846.0, 2828.75, 2801.75, 2784.25, 450.25, 2800.25, 2870.5, 2819.0, 2856.5, 2820.75, 1669.5, 2870.0, 2838.25, 2776.75, 2808.5, 2838.25, 1715.75, 2835.25, 2817.0, 2813.0, 2861.25, 2808.25, 2835.25, 2868.5, 2845.5, 2856.5, 2802.75, 2815.0, 2876.25, 1110.25, 2788.25, 2820.75, 2832.5, 2832.75, 2857.0, 2839.75, 389.75, 2876.25, 2848.5, 2835.25, 2257.0, 2837.25, 2809.75, 2822.75, 2829.5, 2840.0, 2851.25, 2813.0, 501.0, 2811.75, 2841.0, 2846.0, 2

In [20]:
# Accuarcy of the numerical predictions
def getAccuracyNumeric(actual,predicted):
    error=0
    for i in range(len(predicted)):
        error+=pow((actual[i]-predicted[i]),2)
    error=error/len(predicted)-1
    return 100-error

In [21]:
print('Accuracy from the code: {:^0.2f}'.format(getAccuracyNumeric(Y_test,output),2))

Accuracy from the code: -73406.38


In [23]:
print ("Y_test : ", Y_test)

Y_test :  0      2822
1      2863
2      2817
3      2808
4      2788
       ... 
245    2811
246    2874
247    2815
248    2765
249    2832
Name: price, Length: 250, dtype: int64


In [22]:
vals = Y_test.values
for i in range(len(output)):
    print (vals[i], output[i])

554 553.8
554 523.6
402 423.8
554 523.4
2759 1806.4
351 369.2
402 389.6
554 524.0
553 553.8
553 513.8
2757 1876.2
403 410.8
405 401.4
404 422.6
404 391.4
402 423.8
402 413.8
403 422.6
554 523.4
339 369.2
327 423.6
554 510.4
2759 2758.8
552 918.0
351 374.6


In [23]:
print (output[0]-Y_test[0])

-0.20000000000004547


In [26]:
print (mean_absolute_error(Y_test,output) )

108.14399999999999
